# Evaluation criteria

The goal of this assignment is to get a view on your hands-on "data engineering" skills.  
At our company, our data scientists and engineers collaborate on projects.  
Your main focus will be creating performant & robust data flows.  
For a take-home-assignment, we cannot grant you access to our infrastructure.  
The assignement below measures your proficiency in general programming, data science & engineering tasks using python.  
Completion should not take more than half a day.

**We expect you to be proficient in:**
 * SQL queries (Sybase IQ system)
 * ETL flows (In collaboration with existing teams)
 * General python to glue it all together
 * Python data science ecosystem (Pandas + SKlearn)
 
**In this exercise we expect you to demonstrate your ability to / knowledge of:**
 * Building a data science runtime
 * PEP8 / Google python styleguide
 * Efficiently getting the job done
 * Choose meaningfull names for variables & functions
 * Writing maintainable code (yes, you might need to document some steps)
 * Help a data scientist present interactive results.
 * Offer predictions via REST api

# Setting-up a data science workspace

We allow you full freedom in setting up a data science runtime.  
The main objective is having a runtime where you can run this notebook and the code you will develop.  
You can choose for a local setup on your pc, or even a cloud setup if you're up for it.   

**In your environment, you will need things for:**
 * https request
 * python3 (not python2 !!)
 * (geo)pandas
 * interactive maps (e.g. folium, altair, ...)
 * REST apis
 
**Deliverables we expect**:
 * notebook with the completed assignment
 * list of packages for your runtime (e.g. yml or txt file)
 * evidence of a working API endpoint

# Importing packages

We would like you to put all your import statements here, together in 1 place.  
Before submitting, please make sure you remove any unused imports :-)  

In [1]:
import pandas as pd
import requests
import unittest
import re
import lightgbm
import joblib
import geopandas as gpd
import math
import folium

# Data ingestion exercises

## Getting store location data from an API

**Goal:** Obtain a pandas dataframe  
**Hint:** You will need to normalise/flatten the json, because it contains multiple levels  
**API call:** https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places  

In [23]:
import requests
import pandas as pd

def get_clp_places(url):
    """
    Retrieves data from a specified API endpoint and normalizes the JSON response into a flattened pandas DataFrame.

    Parameters:
    - url (str): The URL of the API endpoint to fetch data from.

    Returns:
    - pd.DataFrame: A flattened pandas DataFrame containing the normalized data from the API, or None if the request was not successful.
    """
    # Making the API call
    response = requests.get(url)
    
    # Checking if the request was successful (status code 200)
    if response.status_code == 200:
        # Converting the JSON response to a list of dictionaries
        data = response.json()

        # Normalizing the nested JSON data
        df = pd.json_normalize(data)
        
        return df
    else:
        # Printing an error message if the request was not successful
        print(f"Error: {response.status_code}")
        return None

# Calling the function to get the DataFrame
df_clp = get_clp_places("https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places")

# Displaying the first 10 rows of the DataFrame
if df_clp is not None:
    print(df_clp.head(10))

   placeId                 commercialName branchId sourceStatus  \
0      902                AALST (COLRUYT)     4156           AC   
1      946               AALTER (COLRUYT)     4218           AC   
2      950             AARSCHOT (COLRUYT)     4222           AC   
3      886            ALSEMBERG (COLRUYT)     4138           AC   
4      783                 AMAY (COLRUYT)     3853           AC   
5      650              ANDENNE (COLRUYT)     3596           AC   
6      744  ANDERLECHT (VEEWEYDE) COLRUYT     3759           AC   
7      669   ANDERLECHT (HERBETTELN) COLR     3620           AC   
8      448            ANDERLUES (COLRUYT)     3074           AC   
9      681                  ANS (COLRUYT)     3644           AC   

       sellingPartners                  handoverServices  \
0  [QUALITY, 3RDPARTY]  [CSOP_ORDERABLE, PREPAID_PARCEL]   
1  [QUALITY, 3RDPARTY]  [CSOP_ORDERABLE, PREPAID_PARCEL]   
2  [QUALITY, 3RDPARTY]  [CSOP_ORDERABLE, PREPAID_PARCEL]   
3  [QUALITY, 3RDPARTY]

### Quality checks

We would like you to add several checks on this data based on these constraints:  
 * records > 200
 * latitude between 49 and 52
 * longitude between 2 and 7
 
We dont want you to create a full blown test suite here, we're just gonna use 'asserts' from unittest

In [4]:
tc = unittest.TestCase('__init__')

In [ ]:
# testing if the number of records is greater than 200
tc.assertGreater(len(df_clp), 200, "Number of records is less than 200.")

In [24]:
#testing latitude between 49 & 52
for latitude in df_clp['geoCoordinates.latitude']:
    tc.assertGreaterEqual(latitude, 49, "Latitude is less than 49.")
    tc.assertLessEqual(latitude, 52, "Latitude is greater than 52.")

In [26]:
#testing longitude between 2 & 7
for longitude in df_clp['geoCoordinates.longitude']:
    tc.assertGreaterEqual(longitude, 2, "Longitude is less than 2.")
    tc.assertLessEqual(longitude, 7, "Longitude is greater than 7.")

In [28]:
#testing if place_id is integer
tc.assertEqual(df_clp['placeId'].dtype, 'int64')

# Testing if Latitude lies between 0 & 90, longitude between -180 & +180
tc.assertTrue(
    ((0 <= df_clp['geoCoordinates.latitude']) & (df_clp['geoCoordinates.latitude'] <= 90)).all() and
    ((-180 <= df_clp['geoCoordinates.longitude']) & (df_clp['geoCoordinates.longitude'] <= 180)).all(),
    "Latitude or longitude values are outside the valid range."
)

# testing if branchd is numeric
tc.assertTrue(pd.to_numeric(df_clp['branchId'], errors='coerce').notna().all())

# Checking 'isActive' is a boolean
tc.assertTrue(df_clp['isActive'].isin([True, False]).all())

# Checking for valid country codes
valid_country_codes = {'BE', 'FR', 'LU'}
tc.assertTrue(df_clp['address.countryCode'].isin(valid_country_codes).all(), "Entries have invalid country codes.")

#testing routeurl & moreinfoUrl
url_pattern = re.compile(r'https?://\S+')
tc.assertTrue(df_clp['moreInfoUrl'].str.match(url_pattern).all())
tc.assertTrue(df_clp['routeUrl'].str.match(url_pattern).all())

# Checking the format of dates in 'placeSearchOpeningHours'
date_pattern = re.compile(r'\d{2}-\d{2}-\d{4}')
tc.assertTrue(df_clp['placeSearchOpeningHours'].apply(lambda x: all('date' in entry and re.match(date_pattern, entry['date']) for entry in x)).all())


### Feature creation

Create a new column "antwerpen" which is 1 for all stores in Antwerpen (province) and 0 for all others 

In [35]:
#converting postal code to integer 
df_clp['address.postalcode'] = df_clp['address.postalcode'].astype(int)

#Creating a new feature antwerpen based on the range of postal code
#wikipedia mentions post codes for Antwerp province range between 2000 to 2660 (https://en.wikipedia.org/wiki/Antwerp)
df_clp['antwerpen'] = df_clp['address.postalcode'].apply(lambda x: 1 if 2000 <= x < 2661 else 0)

# Display the counts of values in the 'antwerpen' column
print(df_clp['antwerpen'].value_counts())



antwerpen
0    229
1     30
Name: count, dtype: int64


## Predict used car value

A datascientist in our team made a basic model to predict car prices.  
The model was saved to disk ('lgbr_cars.model') using joblib's dump fuctionality.  
Documentation states the model is a LightGBM Regressor, trained using the sk-learn api.  

**As engineer, your task it to expose this model as REST-api.** 

First, retrieve the model via the function below.  
Change the path according to your setup.  

In [9]:
# Function to load the pretrained model from a local path
def retrieve_model(path):
    if len(path.strip())==0:
        raise ValueError("path should not be empty")
    return joblib.load(path)

# Calling the retrieve_model function using the pretrained model in local directory
lgbr_cars = retrieve_model("lgbr_cars.model")

tc.assertEqual(str(type(lgbr_cars)),"<class 'lightgbm.sklearn.LGBMRegressor'>", type(lgbr_cars))

Now you have your trained model, lets do a functional test based on the parameters below.  
You have to present the parameters in this order.  

* vehicleType: coupe
* gearbox: manuell
* powerPS: 190
* model: NaN
* kilometer: 125000
* monthOfRegistration: 5 
* fuelType: diesel
* brand: audi

Based on these parameters, you should get a predicted value of 14026.35068804
However, the model doesnt accept string inputs, see the integer encoding below:

In [11]:
# Integer encoded input to test the model by predicting
model_test_input = [[3,1,190,-1,125000,5,3,1]]

In [12]:
#Function to predict the car price from the give input values and using the given model
def make_prediction(trained_model, single_input):
    if str(type(trained_model))!="<class 'lightgbm.sklearn.LGBMRegressor'>" :
        raise ValueError ("Please use a valid LGBMRegressor model")
    if len(single_input)>1:
        raise ValueError ("Please use a single input value")
    return (trained_model.predict(single_input))[0]

predicted_value = make_prediction(lgbr_cars, model_test_input)

tc.assertAlmostEqual(predicted_value, 14026.35, places=2)

Now you got this model up and running, we want you to **expose it as a rest api.**  
We don't expect you to set up any authentication.  
We're not looking for beautiful inputs, just make it work.  
**Building this endpoint should NOT be done in a notebook, but in proper .py file(s)**

Once its up and running, use it to predict the following input:
* [-1,1,0,118,150000,0,1,38] ==> prediction should be 13920.70

## Geospatial data exercise
The goal of this exercise is to read in some data from a shape file and visualize it on a map
- The map should be dynamic. I want to zoom in and out to see more interesting aspects of the map
- We want you to visualize the statistical sectors within a distance of 2KM of your home location.

Specific steps to take:
- Read in the shape file
- Transform to WGS coordinates
- Create a distance function (Haversine)
- Create variables for home_lat, home_lon and perimeter_distance
- Calculate centroid for each nis district
- Calculate the distance to home for each nis district centroid 
- Figure out which nis districts are near your home
- Create dynamic zoomable map
- Visualize the nis districts near you (centroid <2km away), on the map


In [14]:
# part 1: Reading in the data

#Reading the shape file
# get this file from https://statbel.fgov.be/sites/default/files/files/opendata/Statistische%20sectoren/sh_statbel_statistical_sectors_20200101.shp.zip 
df = gpd.read_file('sh_statbel_statistical_sectors_20200101.shp')

# Calculate centroid coordinates after changing the projection
# Use the Mercator projection (EPSG=3395) for distance calculations
df['centroid_lon'] = df.to_crs(epsg=3395).geometry.centroid.x
df['centroid_lat'] = df.to_crs(epsg=3395).geometry.centroid.y

In [16]:
# Define a home location and a perimeter distance. Here, I have taken Zwijnaarde
home_lat = 51.0031080246
home_lon = 3.71061933068
perimeter_distance = 2 # km

In [17]:
# Harversine distance function
""" 
Formula of Haversine used here

a = sin²(φB - φA/2) + cos φA * cos φB * sin²(λB - λA/2)
c = 2 * atan2( √a, √(1−a) )
d = R ⋅ c
φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km)
Coordinates in decimal degrees.
"""

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two sets of latitude and longitude coordinates.

    Parameters:
    - lat1, lon1: Latitude and longitude of the first point
    - lat2, lon2: Latitude and longitude of the second point

    Returns:
    - distance: Haversine distance in kilometers
    """
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Calculate the differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Calculate the distance
    distance = R * c
    
    return distance


Next, implement some sanity checks for your distance function 

In [18]:
# implement sanity checks here
# Checking the distance between Hyderabad & Brussels.
distance_check = haversine(50.8476, 4.3572, 17.4065, 78.4772)
print(f"Distance between Hyderabad & Brussels: {distance_check:.2f} km")

Distance between Hyderabad & Brussels: 7407.78 km


Now, create a dynamical map 

In [ ]:
# implementation of the map goes here

In [22]:
# Identify districts near Zwijnaarde Hekers using the Haversine distance function
df['distance_haversine'] = df.apply(lambda row: haversine(home_lat, home_lon, row['centroid_lat'], row['centroid_lon']), axis=1)
near_home = df[df['distance_haversine'] <= perimeter_distance]


In [24]:
#point as input: zwijnaarde
# Create a folium map centered at the zwijnaarde Hekers location
zwijnaarde = folium.Map(location = [home_lat,home_lon],zoom_start=20)

# Put the nearby districts to our input on the map
folium.GeoJson(near_home.geometry).add_to(zwijnaarde)

#display the map
display(zwijnaarde)